In [ ]:
import os 
import sys
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(__file__)), 'hsa_dpo/models/llava-v1_5'))
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0"
model_base="/mnt/workspace/zhidaonas/wenyi/ckpts/llava-v1.5-13b"
# model_base="/mnt/workspace/wenyi/ckpts/LLaVA-RLHF/sft_model"
# model_path="/mnt/workspace/zhidaonas/wenyi/ckpts/hadpo/2024_04_06_21_36_35_llava_v1.5-13B-lora-silkie-llava_baseline_34b_cho_rej_normal_bz16"
model_path="/mnt/workspace/zhidaonas/wenyi/visual_mcts/pretrain_model/HSA-DPO_llava_v1.5-13B-lora"
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model
tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=model_base,
    model_name=get_model_name_from_path(model_path)
)

You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading LLaVA from base model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/home/pai/envs/llava/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [1:00:32<00:00, 1210.98s/it]


Loading additional LLaVA weights...
Loading LoRA weights...
Merging LoRA weights...
Model is loaded...


In [9]:
import argparse
import torch

from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
    IMAGE_PLACEHOLDER,
)
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import (
    process_images,
    tokenizer_image_token,
    get_model_name_from_path,
)

from PIL import Image

import requests
from PIL import Image
from io import BytesIO
import re


def image_parser(args):
    out = args.image_file.split(args.sep)
    return out


def load_image(image_file):
    if image_file.startswith("http") or image_file.startswith("https"):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")
    return image


def load_images(image_files):
    out = []
    for image_file in image_files:
        image = load_image(image_file)
        out.append(image)
    return out


def eval_model(args):
    # Model
    disable_torch_init()
    qs = args.query
    image_token_se = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN
    if IMAGE_PLACEHOLDER in qs:
        if model.config.mm_use_im_start_end:
            qs = re.sub(IMAGE_PLACEHOLDER, image_token_se, qs)
        else:
            qs = re.sub(IMAGE_PLACEHOLDER, DEFAULT_IMAGE_TOKEN, qs)
    else:
        if model.config.mm_use_im_start_end:
            qs = image_token_se + "\n" + qs
        else:
            qs = DEFAULT_IMAGE_TOKEN + "\n" + qs
    model_name = args.model_name
    # print(model_name)
    if "llama-2" in model_name.lower():
        conv_mode = "llava_llama_2"
    elif "mistral" in model_name.lower():
        conv_mode = "mistral_instruct"
    elif "v1.6-34b" in model_name.lower():
        conv_mode = "chatml_direct"
    elif "v1" in model_name.lower():
        conv_mode = "llava_v1"
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"
    else:
        conv_mode = "llava_v0"

    if args.conv_mode is not None and conv_mode != args.conv_mode:
        print(
            "[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}".format(
                conv_mode, args.conv_mode, args.conv_mode
            )
        )
    else:
        args.conv_mode = conv_mode

    conv = conv_templates[args.conv_mode].copy()
    conv.append_message(conv.roles[0], qs)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()

    image_files = image_parser(args)
    images = load_images(image_files)
    image_sizes = [x.size for x in images]
    images_tensor = process_images(
        images,
        image_processor,
        model.config
    ).to(model.device, dtype=torch.float16)

    input_ids = (
        tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors="pt")
        .unsqueeze(0)
        .cuda()
    )

    with torch.inference_mode():
        output_ids = model.generate(
            input_ids,
            images=images_tensor,
            image_sizes=image_sizes,
            do_sample=True if args.temperature > 0 else False,
            temperature=args.temperature,
            top_p=args.top_p,
            num_beams=args.num_beams,
            max_new_tokens=args.max_new_tokens,
            use_cache=True,
        )

    outputs = tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
    # print("*****")
    # print("outputs:"+outputs)
    # print("*****")
    return outputs
import torch
torch.backends.cudnn.enabled = False
import argparse
import os
import json
from tqdm import tqdm
import shortuuid
import sys
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, process_images, get_model_name_from_path

from PIL import Image
import math
# image_file = "/mnt/workspace/wenyi/benchmark/Amber/image/{}"
# image_file = "/mnt/workspace/wenyi/benchmark/mmhal/images/{}"
image_file = "/mnt/workspace/zhidaonas/wenyi/benchmark/datasets/MSCOCO_2014/val/COCO_val2014_{}"
# image_file = "/mnt/workspace/wenyi/benchmark/datasets/ObjHal/{}"

# question_file = "/mnt/workspace/wenyi/benchmark/Amber/AMBER/data/query/query_generative.json"
question_file = "/mnt/workspace/zhidaonas/wenyi/benchmark/LLaVA_bench/origin_desc/dpo_v1.jsonl"
# question_file = "/mnt/workspace/wenyi/benchmark/Amber/AMBER/results/dpo/llavav3.json"
# question_file = "/mnt/workspace/wenyi/benchmark/pope/adversarial/coco_pope_adversarial.jsonl"
# question_file = "/mnt/workspace/wenyi/benchmark/obj_halbench/all_cap/cap_file11/obj_halbench_300_llava1_5_13B_dpo.jsonl"
# question_file = "/mnt/workspace/wenyi/benchmark/mmhal/response_template.json"
with open(question_file, "r") as f:
    question = [json.loads(l) for l in f]
    # question = json.load(f)
print(len(question))
print(question[0])
from tqdm import tqdm

for idx, item in enumerate(question):
    # if idx<=270:
    #     continue
    print(idx)
    prompt_text = f"""{item["prompt"]}"""
    args = type('Args', (), {
        "model_path": model_path,
        "model_base": model_base,
        "model_name": get_model_name_from_path(model_path),
        "query": prompt_text,
        "conv_mode": None,
        "image_file": image_file.format(item["image"]),
        "sep": ",",
        "temperature": 0.7,
        "top_p": 0.95,
        "num_beams": 2,
        "max_new_tokens": 1024,
        "repetition_penalty": 1.1
    })()
    # item["prompt"] = item["text"]
    item["text"] = eval_model(args)
    item["response"] = item["text"]
    print(prompt_text)
    print(item["text"])
    print("---------")
    # print(item["response"])
    # print("***2222")
    # print(item["response"])
    # print("***2222")

# with open("/mnt/workspace/wenyi/benchmark/obj_halbench/all_cap/cap_file31/obj_halbench_300_llava_dpo12k.jsonl", "w") as f:
#     for item in question:
#         f.write(json.dumps(item)+"\n")

with open("/mnt/workspace/zhidaonas/wenyi/benchmark/LLaVA_bench/origin_desc/dpo_v25_llava_dpo_12k_rej_normal_epock2in2.jsonl", "w") as f:
    for item in question:
        f.write(json.dumps(item)+"\n")

90
{'image': '000000441147.jpg', 'question_id': 0, 'prompt': 'What is the color of the two suitcases in the image?', 'text': 'Yes, the two suitcases in the image are black.', 'model_id': '/mnt/workspace/wenyi/ckpts/llava-v1.5-13b', 'metadata': {}}
0


/home/pai/envs/llava/lib/python3.10/site-packages/transformers/generation/utils.py:1295: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


What is the color of the two suitcases in the image?
The color of the two suitcases in the image is black.
---------
1
Analyze the image in a comprehensive and detailed manner.
The image features an old-fashioned leather suitcase, which appears to be a vintage piece. The suitcase has a tag attached to it, possibly indicating its origin or ownership. The suitcase is sitting on a surface, and it is the main focus of the image. The suitcase's design and material give it a classic and nostalgic appearance.
---------
2
What potential factors could make these suitcases valuable?
The suitcases in the image appear to be vintage or antique, which can make them valuable due to their age, rarity, and historical significance. Vintage suitcases often have unique designs, materials, and craftsmanship that are no longer commonly found in modern luggage. Collectors and enthusiasts might be interested in these suitcases for their aesthetic appeal, historical context, or as a representation of a bygone 